In [1]:
import os
import pandas as pd
import numpy as np
import wrds
from random import *
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import matplotlib.pyplot as plt
import pandas_datareader
import datetime
import warnings
from scipy.optimize import fmin
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
data_folder = '/Users/chengxinxiangye/Desktop/Quant Asset/' 

In [3]:
#Load Bond Data from folder
CRSP_Bonds = pd.read_pickle(data_folder + 'bonds_PS2.pkl')

In [4]:
def PS2_Q1(df):
    # Restrict time interval of Bond
    df = df[(df['date'].dt.year>1925) & (df['date'].dt.year<2024)].copy()
    
    # Sort data
    df = df.sort_values(['idCRSP', 'date']).reset_index(drop=True).copy()
    # Drop missing returns
    df = df[df['ret'].notna()].reset_index(drop=True).copy()

    # Calculate lagged market equity
    df['Lme'] = df[['idCRSP','me']].groupby('idCRSP').shift(1)
    
    # Drop missing market quity after we lagged
    df = df[df['Lme'].notna()].reset_index(drop=True).copy()

    # Calculate value weight using lagged market equity 
    df = df.merge(df[['idCRSP','date','Lme']].groupby(['date']).sum()\
                        .reset_index().rename(columns={"Lme":"LmeTotal"}),on=['date'],
                        how='left')

    # Calculate monthly value weighted return for each Bond
    df['vwret'] = df['ret'] * (df['Lme'] / df['LmeTotal']) 

    # Calculate monthly vw, lagged return by sum monthly per bond return
    # Calculate equal weighted by simply take average of return in one month for all bonds
    df = df[['date', 'vwret', 'Lme', 'ret']].groupby(['date'])\
                            .agg({'vwret': 'sum','Lme': 'sum','ret':'mean'}).reset_index()\
                            .rename(columns={'vwret': 'Bond_Vw_Ret', 'Lme': 'Bond_lag_MV', 'ret' : "Bond_Ew_Ret" })

    # Creat Year Variable for output
    df['Year'] = df['date'].dt.year

    # Creat Month Variable for output
    df['Month'] = df['date'].dt.month

    # Output only required columns
    result = df[['Year','Month','Bond_lag_MV',"Bond_Ew_Ret",'Bond_Vw_Ret']].copy()

    return result

In [5]:
Monthly_CRSP_Bonds = PS2_Q1(CRSP_Bonds)

In [6]:
print(Monthly_CRSP_Bonds.dtypes)
print(Monthly_CRSP_Bonds.head(5))

Year             int32
Month            int32
Bond_lag_MV    float64
Bond_Ew_Ret    float64
Bond_Vw_Ret    float64
dtype: object
   Year  Month  Bond_lag_MV  Bond_Ew_Ret  Bond_Vw_Ret
0  1926      2      19502.0     0.003621     0.003844
1  1926      3      18884.0     0.003812     0.003588
2  1926      4      18736.0     0.003740     0.006393
3  1926      5      19227.0     0.002146     0.002629
4  1926      6      18892.0     0.004319     0.003755


In [7]:
#Load Stock and T-bill Data from folder
Monthly_CRSP_Riskless = pd.read_pickle(data_folder + 'rf_PS2.pkl')
Monthly_CRSP_Stocks = pd.read_pickle(data_folder + 'mkt_csrp_PS2.pkl')

In [8]:
def PS2_Q2(Stocks, Bonds, Rf):
    # Sort data
    Stocks = Stocks.sort_values(['date']).reset_index(drop=True).copy()
    Rf = Rf.sort_values(['date']).reset_index(drop=True).copy()

    # Adjust data to be lagged and unit with million
    Stocks['Stock_lag_MV'] = (Stocks['Stock_lag_MV'] * 1e-3).shift(1)
    Stocks = Stocks[Stocks['Stock_lag_MV']>0].copy()
    
    # First merge stock data and riskfree data because both have data column
    Q1 = pd.merge(Stocks,Rf,on=['date'],how='outer').copy()
    
    # Then create Year and Month column to merge with Bonds data in Q1
    Q1['Year'] = Q1['date'].dt.year
    Q1['Month'] = Q1['date'].dt.month
    
    # Merge Bond and other two data with Month and Year
    Q2 = pd.merge(Q1,Bonds,on=['Month','Year'],how='outer').copy()
    
    # Restrict time interval 
    Q2 = Q2[(Q2['date'].dt.year>1925) & (Q2['date'].dt.year<2024)].copy()
    
    # Sort date by date and drop missing values
    Q2 = Q2.sort_values(['date']).reset_index(drop=True).dropna().copy()
    
    # Calculate excess value weighted return for bonds and stocks(Use rf 30 since we calculated vw return monthly)
    Q2['Stock_Excess_Vw_Ret'] = Q2['Stock_Vw_Ret'] - Q2['rf30']
    Q2['Bond_Excess_Vw_Ret'] = Q2['Bond_Vw_Ret'] - Q2['rf30']
    
    # Output only required columns
    Q2 = Q2[['Year','Month','Stock_lag_MV','Stock_Excess_Vw_Ret','Bond_lag_MV','Bond_Excess_Vw_Ret']].copy()
    
    return Q2

In [9]:
Monthly_CRSP_Univers = PS2_Q2(Monthly_CRSP_Stocks,Monthly_CRSP_Bonds,Monthly_CRSP_Riskless)
print(Monthly_CRSP_Univers.dtypes)
print(Monthly_CRSP_Univers.head(5))

Year                     int32
Month                    int32
Stock_lag_MV           float64
Stock_Excess_Vw_Ret    float64
Bond_lag_MV            float64
Bond_Excess_Vw_Ret     float64
dtype: object
   Year  Month  Stock_lag_MV  Stock_Excess_Vw_Ret  Bond_lag_MV  \
1  1926      2    27624.2408            -0.035814      19502.0   
2  1926      3    26752.0641            -0.066780      18884.0   
3  1926      4    25083.1734             0.033957      18736.0   
4  1926      5    25886.7438             0.011753      19227.0   
5  1926      6    26187.8079             0.050205      18892.0   

   Bond_Excess_Vw_Ret  
1            0.001076  
2            0.000810  
3            0.003321  
4            0.002287  
5            0.000296  


In [10]:
def PS2_Q3(Q2):
    Q3 = Q2.copy()

    '''
    VW portfolio: Value-weighted portfolio is a market portfolio weighted by total market capitalization 
    and rebalanced monthly to maintain value weights.
    '''
    
    # weights = lagged market value / total market value
    Q3['Stock_W1'] = Q3['Stock_lag_MV'] / (Q3['Stock_lag_MV'] + Q3['Bond_lag_MV'])
    Q3['Bond_W1'] = Q3['Bond_lag_MV'] / (Q3['Stock_lag_MV'] + Q3['Bond_lag_MV'])
    
    # Excess value weighted return = weight of bond * bond excess return + weight of stock * stock excess return
    Q3['Excess_Vw_Ret'] = Q3['Stock_W1'] * Q3['Stock_Excess_Vw_Ret'] + Q3['Bond_W1'] * Q3['Bond_Excess_Vw_Ret']

    # 60% weight of stock and 40% weight of bond
    Q3['Excess_60_40_Ret'] = 0.6 * Q3['Stock_Excess_Vw_Ret'] + 0.4 * Q3['Bond_Excess_Vw_Ret']

    '''
    Unlevered_RP:
    Weight for assets are inverse volatility times k;
    K is set to be inverse of sum of inversed volatility for each asset;
    Volatility calculated using rolling window in 3 year of excess return until t-1
    '''
    Q3['Stock_inverse_sigma_hat'] = 1/(Q3['Stock_Excess_Vw_Ret'].shift(1).rolling(window=12*3).std())
    Q3['Bond_inverse_sigma_hat'] = 1/(Q3['Bond_Excess_Vw_Ret'].shift(1).rolling(window=12*3).std())
    Q3['Unlevered_k'] = 1/(Q3['Stock_inverse_sigma_hat'] + Q3['Bond_inverse_sigma_hat'])
    Q3['Excess_Unlevered_RP_Ret'] = Q3['Unlevered_k'] * Q3['Stock_inverse_sigma_hat'] * Q3['Stock_Excess_Vw_Ret'] + Q3['Unlevered_k'] * Q3['Bond_inverse_sigma_hat'] * Q3['Bond_Excess_Vw_Ret'] 

    # levered_RP
    # Benchmark volatility
    Q3['BM_Vol'] = Q3['Excess_Vw_Ret'].std()
    # Define a function to find k that minimize difference between vol of portfolio and benchmark
    def f(k, Q3):
        levered_vol = (k * (Q3['Stock_inverse_sigma_hat'] * Q3['Stock_Excess_Vw_Ret'] + 
                           Q3['Bond_inverse_sigma_hat'] * Q3['Bond_Excess_Vw_Ret'])).std()
        differences = (Q3['BM_Vol'] - levered_vol).dropna()
        return np.sum(differences ** 2)
    
    #find the target k
    result = fmin(f, [1], (Q3,),  xtol=1e-10, ftol=1e-10)

    # Define 
    Q3['Levered_k'] = result[0]
    Q3['Excess_Levered_RP_Ret'] = Q3['Levered_k'] * Q3['Stock_inverse_sigma_hat'] * Q3['Stock_Excess_Vw_Ret'] + Q3['Levered_k'] * Q3['Bond_inverse_sigma_hat'] * Q3['Bond_Excess_Vw_Ret']

    
    # only keep requried

    Q3 = Q3[["Year", "Month", "Stock_Excess_Vw_Ret", "Bond_Excess_Vw_Ret", "Excess_Vw_Ret",\
             "Excess_60_40_Ret", "Stock_inverse_sigma_hat", "Bond_inverse_sigma_hat",\
             "Unlevered_k", "Excess_Unlevered_RP_Ret", "Levered_k", "Excess_Levered_RP_Ret"]].copy()

    return Q3

In [11]:
Port_rets = PS2_Q3(Monthly_CRSP_Univers)
print(Port_rets.dtypes)
print(Port_rets.head(5))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 37
         Function evaluations: 74
Year                         int32
Month                        int32
Stock_Excess_Vw_Ret        float64
Bond_Excess_Vw_Ret         float64
Excess_Vw_Ret              float64
Excess_60_40_Ret           float64
Stock_inverse_sigma_hat    float64
Bond_inverse_sigma_hat     float64
Unlevered_k                float64
Excess_Unlevered_RP_Ret    float64
Levered_k                  float64
Excess_Levered_RP_Ret      float64
dtype: object
   Year  Month  Stock_Excess_Vw_Ret  Bond_Excess_Vw_Ret  Excess_Vw_Ret  \
1  1926      2            -0.035814            0.001076      -0.020548   
2  1926      3            -0.066780            0.000810      -0.038812   
3  1926      4             0.033957            0.003321       0.020858   
4  1926      5             0.011753            0.002287       0.007719   
5  1926      6             0.050205            0.000296   

In [12]:
def calulation(data):
    AM = round(data.mean()*1200,2) # in percentage
    # Assume our population mean is 0 to calculate t-stat
    T = round((data.mean())/(data.std()/np.sqrt(len(data))),2)
    AS = round(data.std()*np.sqrt(12)*100,2) # in percentage
    SR = round(AM/AS,2)
    SK = round(data.skew(),2)
    KT = round(data.kurtosis(),2)
    return [AM,T,AS,SR,SK,KT]

In [17]:
def PS2_Q4(Q3):
    # Specify requried column names and row names
    colname = ['Excess Return(%)','t-stat', 'Volatility(%)',\
               'Sharpe Ratio', 'Skewness','Excess Kurtosis']
    rowname = ["CRSP stocks", "CRSP bonds", "Value-weighted portfolio",\
                "60/40 portfolio", "RP, unlevered", "RP", "RP minus value-weighted",\
                "RP minus 60/40"]
    # Creat a dataframe with row,colnames
    df = pd.DataFrame(np.nan, index=rowname, columns=colname)
    
    # Make sure only include data from Jan 1929 to June 2010
    start = np.where((Q3['Year'] == 1929) & (Q3['Month'] == 1))[0][0]
    end = np.where((Q3['Year'] == 2010) & (Q3['Month'] == 6))[0][0]
    Q3 = Q3[start:(end+1)].copy()

    # Calculate data to each column in dataframe
    df.loc["CRSP stocks"] = calulation(Q3['Stock_Excess_Vw_Ret'])
    df.loc["CRSP bonds"] = calulation(Q3["Bond_Excess_Vw_Ret"])
    df.loc["Value-weighted portfolio"] = calulation(Q3['Excess_Vw_Ret'])
    df.loc["60/40 portfolio"] = calulation(Q3['Excess_60_40_Ret'])
    df.loc["RP, unlevered"] = calulation(Q3['Excess_Unlevered_RP_Ret'])
    df.loc["RP"] = calulation(Q3['Excess_Levered_RP_Ret'])
    df.loc["RP minus value-weighted"] = calulation(Q3['Excess_Levered_RP_Ret'] - Q3['Excess_Vw_Ret'])
    df.loc["RP minus 60/40"] = calulation(Q3['Excess_Levered_RP_Ret'] - Q3['Excess_60_40_Ret'])
    return(df)

In [18]:
result = PS2_Q4(Port_rets)
result

,Excess Return(%),t-stat,Volatility(%),Sharpe Ratio,Skewness,Excess Kurtosis
CRSP stocks,6.71,3.16,19.15,0.35,0.22,7.84
CRSP bonds,1.40,4.32,2.93,0.48,0.23,4.19
Value-weighted portfolio,3.55,2.62,12.25,0.29,-0.56,4.60
60/40 portfolio,4.59,3.54,11.69,0.39,0.23,7.58
"RP, unlevered",2.10,5.08,3.72,0.56,0.07,2.63
RP,6.83,5.02,12.28,0.56,-0.41,2.00
RP minus value-weighted,3.32,3.17,9.47,0.35,0.07,3.58
RP minus 60/40,2.28,2.30,8.95,0.25,-0.69,6.31
